In [ ]:
import sys
sys.path.append("..")
from app.agent import ChaosStar

agent = await ChaosStar.create(llm_name="anthropic", name="ChaosStar", next_step_prompt="test first", max_steps=2)
result = await agent.run("Hello, Claude")

In [ ]:
import sys
sys.path.append("..")
from app.agent import StreamChaosStar

agent = await StreamChaosStar.create(llm_name="anthropic", name="ChaosStarStream")

In [ ]:
import nest_asyncio
nest_asyncio.apply()

import sys
import asyncio

if sys.platform.startswith("win"):
    asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())
sys.path.append("..")
from app.agent import StreamChaosStar

agent = None
prompt = """
我正在为一家初创公司制定2025年的市场进入策略。公司开发了一款基于AI的个人健康监测设备，类似智能手表但专注医疗级数据采集。我需要你：

分析当前可穿戴医疗设备市场的竞争格局和趋势
识别目标市场和用户画像
评估进入壁垒和监管要求
制定差异化定位策略
估算初期市场投入预算

请用结构化思考方式分析，包括数据支撑、风险评估和具体行动建议。
"""

async def main():
    global agent
    agent = await StreamChaosStar.create(llm_name="anthropic", name="ChaosStarStream")
    async for res in agent.run(prompt, stream=True):
        print(res)
    await agent.cleanup()

await main()

In [ ]:
import sys
sys.path.append("..")

from app.llm import LLM

claude = LLM.for_config("anthropic")

In [ ]:
import requests
import time

def test_google_with_proxy():
    """Test accessing Google with proxy"""
    
    proxy = "http://127.0.0.1:7897" # Change to your own proxy if needed
    proxies = {
        'http': proxy,
        'https': proxy
    }
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    url = "https://www.google.com"
    
    try:
        print(f"Testing connection to {url} with proxy {proxy}")
        start_time = time.time()
        
        response = requests.get(
            url, 
            headers=headers, 
            proxies=proxies, 
            timeout=10
        )
        
        end_time = time.time()
        
        print(f"✅ Success!")
        print(f"Status Code: {response.status_code}")
        print(f"Response Time: {end_time - start_time:.2f}s")
        print(f"Content Length: {len(response.text)} characters")
        print(f"Title: {response.text[response.text.find('<title>'):response.text.find('</title>') + 8] if '<title>' in response.text else 'No title found'}")
        
        return True
        
    except Exception as e:
        print(f"❌ Failed: {e}")
        return False

if __name__ == "__main__":
    test_google_with_proxy()

Testing connection to https://www.google.com with proxy http://127.0.0.1:7897
✅ Success!
Status Code: 200
Response Time: 1.21s
Content Length: 167497 characters
Title: <title>Google</title>


In [ ]:
import sys
sys.path.append("..")
from app.llm import LLM

claude = LLM.for_config("anthropic")

In [ ]:
claude.thinking

In [ ]:
import os

def count_py_lines(root):
    total_lines = 0
    for dirpath, dirnames, filenames in os.walk(root):
        for filename in filenames:
            if filename.endswith('.py'):
                file_path = os.path.join(dirpath, filename)
                try:
                    with open(file_path, 'r', encoding='utf-8') as f:
                        lines = f.readlines()
                        line_count = len([line for line in lines if line.strip()])
                        print(f"{file_path}: {line_count} lines")
                        total_lines += line_count
                except Exception as e:
                    print(f"Could not read {file_path}: {e}")
    print(f"Total lines: {total_lines}")

if __name__ == '__main__':
    root_dir = r'C:\Users\Administrator\Desktop\LLM\ChaosStar'
    count_py_lines(root_dir)

In [ ]:
from mcp.types import ClientRequest, CallToolRequest

params = {'name': 'broswer_use', 'action': 'web_search', 'url': None, 'index': None, 'text': None, 'scroll_amount': None, 'tab_id': None, 'query': 'Elon Musk interesting events 2025', 'goal': None, 'keys': None, 'seconds': None}
method="tools/call"
request = ClientRequest(CallToolRequest(method=method, params=params))
dump = request.model_dump(by_alias=True, mode="json", exclude_none=True)
print(dump)

In [ ]:
import os
import asyncio
import sys
sys.path.append("..")
import tomllib
import httpx
from anthropic import AsyncAnthropic
from app.config import PROJECT_ROOT

file_path = PROJECT_ROOT /"config" / "config.example.test.toml"
with file_path.open("rb") as f:
    llm_config = tomllib.load(f)
api_key = llm_config.get("llm", None).get("anthropic", {}).get("api_key", None)
local_proxy = llm_config.get("llm", None).get("anthropic", {}).get("proxy", None)
proxy_client = httpx.AsyncClient(
    proxy=httpx.Proxy(url=local_proxy),
    timeout=30,
)

client = AsyncAnthropic(
    api_key=api_key,  # This is the default and can be omitted
    base_url="https://api.anthropic.com",
    http_client=proxy_client
)


async def main() -> None:
    message = await client.messages.create(
        max_tokens=1024,
        messages=[
            {
                "role": "user",
                "content": "Hello, Claude",
            }
        ],
        model="claude-3-5-haiku-20241022",
    )
    print(message.content)


await main()